# How to compute a contrast grid?

In [1]:
from pathlib import Path

import seaborn as sns
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt

## Imports

In [2]:
from applefy.detections.contrast import Contrast

from applefy.utils.file_handling import load_adi_data
from applefy.utils import flux_ratio2mag, mag2flux_ratio

## Data Loading

In [3]:
root_dir = Path("/home/ipa/quanz/user_accounts/mbonse/2021_Metrics/70_results/documentation_demos")

In [4]:
dataset_config = {'file_path': Path('30_data/betapic_naco_lp_stack.hdf5'),
                  'stack_key': "stack_no_planet",
                  'psf_template_key': "flux",
                  'parang_key': 'header_stack/PARANG',
                  'dit_psf_template': 0.02019,
                  'fwhm_size' : 4.2,
                  'dit_science': 0.2}

In [5]:
# we need the psf template for contrast calculation
science_data, angles, raw_psf_template_data = load_adi_data(
    root_dir/dataset_config["file_path"],
    data_tag=dataset_config["stack_key"],
    psf_template_tag=dataset_config["psf_template_key"],
    para_tag=dataset_config["parang_key"])

dit_psf_template = dataset_config["dit_psf_template"]
dit_science = dataset_config["dit_science"]
fwhm = dataset_config["fwhm_size"]

psf_template = raw_psf_template_data[82:-82, 82:-82]
science_data = science_data[:, 55:-55, 55:-55]

## Contrast Grid Code

In [6]:
contrast_instance = Contrast(
    science_sequence=science_data,
    psf_template=psf_template,
    psf_fwhm_radius=fwhm / 2,
    parang_rad=angles,
    dit_psf_template=dit_psf_template,
    dit_science=dit_science,
    checkpoint_dir=root_dir / Path("70_results/contrast_grid"))

In [7]:
flux_ratios_mag = np.linspace(7.5, 12, 10)
flux_ratios_mag

array([ 7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5, 11. , 11.5, 12. ])

In [8]:
flux_ratios = mag2flux_ratio(flux_ratios_mag)

In [9]:
num_fake_planets = 3

contrast_instance.design_fake_planet_experiments(
    flux_ratios=flux_ratios,
    num_planets=num_fake_planets,
    overwrite=True)

Overwriting existing config files.


## Run Fake planet experiments

In [10]:
from applefy.wrappers.pynpoint import MultiComponentPCAPynPoint, SimplePCAPynPoint
from applefy.utils.file_handling import save_as_fits

In [11]:
components = [5, 10, 20, 30, 50]

In [12]:
algorithm_function = MultiComponentPCAPynPoint(
    num_pcas=components,
    scratch_dir=contrast_instance.scratch_dir,
    num_cpus_pynpoint=1)

In [13]:
contrast_instance.run_fake_planet_experiments(
    algorithm_function=algorithm_function,
    num_parallel=32)

Running fake planet experiments...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 241/241 [00:03<00:00, 69.43it/s]


[DONE]


## Compute the contrast grid

In [14]:
from applefy.utils.photometry import AperturePhotometryMode
from applefy.statistics import TTest, gaussian_sigma_2_fpf, fpf_2_gaussian_sigma

In [15]:
photometry_mode_planet = AperturePhotometryMode("FS", psf_fwhm_radius=fwhm/2, search_area=0.5)
photometry_mode_noise = AperturePhotometryMode("P", psf_fwhm_radius=fwhm/2)

In [16]:
contrast_instance.prepare_contrast_results(
    photometry_mode_planet=photometry_mode_planet,
    photometry_mode_noise=photometry_mode_noise)

In [17]:
ttest_statistic = TTest(1)

In [18]:
contrast_curves_grid, contrast_grids = contrast_instance.compute_contrast_grids(
    statistical_test=ttest_statistic,
    num_cores=32,
    confidence_level_fpf=gaussian_sigma_2_fpf(5),
    num_rot_iter=20,
    safety_margin=2.0,
    pixel_scale=0.02718)

Computing contrast grid for PCA (005 components)
Computing contrast grid with multiprocessing:
................................................................................[DONE]
Computing contrast grid for PCA (010 components)
Computing contrast grid with multiprocessing:
................................................................................[DONE]
Computing contrast grid for PCA (020 components)
Computing contrast grid with multiprocessing:
................................................................................[DONE]
Computing contrast grid for PCA (030 components)
Computing contrast grid with multiprocessing:
................................................................................[DONE]
Computing contrast grid for PCA (050 components)
Computing contrast grid with multiprocessing:
................................................................................[DONE]


## Show the contrast grid

In [19]:
from math import pi
from bokeh.events import Tap
from bokeh import events

from bokeh.models import Rect

from bokeh.plotting import figure, show,  output_file, save
from bokeh.models import ColumnDataSource, CustomJS, Select, Label, Div, Legend, Span
from bokeh.layouts import column, row
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper
from bokeh.models import Slider, RangeSlider
from bokeh.models import TapTool
from bokeh.layouts import gridplot

from bokeh.core.validation import silence
from bokeh.core.validation.warnings import MISSING_RENDERERS
_=silence(MISSING_RENDERERS, True)

In [28]:
# Start the bokeh for jupyter notebooks
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [20]:
contrast_grids_prepared = dict()
all_residual_prepared = dict()

for tmp_components in components:
    tmp_method_name = "PCA (" + str(tmp_components).zfill(3) + " components)"
    print("Prepare results for: " + tmp_method_name)
    
    # 1.) Prepare the contrast Grid
    tmp_raw_grid = contrast_grids[tmp_method_name]
    tmp_grid_list = tmp_raw_grid.unstack().reset_index()
    tmp_grid_list = tmp_grid_list.rename({0: 'fpf'}, axis=1)
    
    # Convert some units
    tmp_grid_list["mag"] = flux_ratio2mag(tmp_grid_list["flux_ratio"])
    tmp_grid_list["fpf"] = fpf_2_gaussian_sigma(tmp_grid_list["fpf"])
    
    # Convert to dict 
    contrast_grids_prepared[str(tmp_components) + "_mag"] = tmp_grid_list["mag"].values
    contrast_grids_prepared[str(tmp_components) + "_fpf"] = tmp_grid_list["fpf"].values
    contrast_grids_prepared[str(tmp_components) + "_fpf_str"] = tmp_grid_list["fpf"].values.astype('<U4')
    contrast_grids_prepared[str(tmp_components) + "_separation [FWHM]"] = tmp_grid_list["separation [FWHM]"].values
    
    # 2.) Prepare the Residuals
    tmp_residuals = residuals = contrast_instance.contrast_results[
        tmp_method_name].residuals
    
    for i, planet_idx in enumerate(["a", "b", "c"]):
        tmp_residuals_position = tmp_residuals[:, :, i, :, :]
        residual_size = tmp_residuals.shape[-1]
        tmp_residuals_position = tmp_residuals_position.reshape(
            -1, residual_size, residual_size)
        
        for idx in range(tmp_residuals_position.shape[0]):
            tmp_idx_str = str(tmp_components) + "_" + str(planet_idx) + "_" + str(idx)
            all_residual_prepared[tmp_idx_str] = tmp_residuals_position[idx]

Prepare results for: PCA (005 components)
Prepare results for: PCA (010 components)
Prepare results for: PCA (020 components)
Prepare results for: PCA (030 components)
Prepare results for: PCA (050 components)


In [21]:
example_grid = contrast_grids["PCA (005 components)"]

x_range_min = example_grid.columns.values[0] - 0.5
x_range_max = example_grid.columns.values[-1] + 0.5

y_range_min = flux_ratio2mag(example_grid.index.values)[0] - 0.25
y_range_max = flux_ratio2mag(example_grid.index.values)[-1] + 0.25

separations = list(example_grid.columns.values.astype(str))
separation_start = separations[4]

contrasts = list(flux_ratio2mag(example_grid.index.values).astype(str))
contrast_start = contrasts[3]

components = [str(i) for i in components]
component_start = "10"

In [22]:
example_list = example_grid.unstack().reset_index()

def sep_flux_2_idx(
    separation_in,
    contrast_in):

    select_1 = example_list["separation [FWHM]"] == float(separation_in)
    select_2 = flux_ratio2mag(example_list["flux_ratio"]) == float(contrast_in)

    return example_list[select_1 & select_2][0].index[0]

In [23]:
start_idx = str(sep_flux_2_idx(separation_start, contrast_start))
start_idx

'43'

In [24]:
residual_selected = {"residual" : [all_residual_prepared[
    component_start + "_a_" + start_idx]]}

In [25]:
contrast_curve_selected = {
    "mag" : contrast_grids_prepared[component_start + "_mag"],
    "fpf" : contrast_grids_prepared[component_start + "_fpf"],
    "fpf_str" : contrast_grids_prepared[component_start + "_fpf_str"],
    "separation [FWHM]" : contrast_grids_prepared[component_start + "_separation [FWHM]"]}

In [26]:
# Create a lookup table for the select widgets
idx_lookup_table = dict()

for tmp_separation in separations:
    for tmp_contrast in contrasts:
        tmp_id = sep_flux_2_idx(tmp_separation, tmp_contrast)
        idx_lookup_table[str(tmp_separation)+ "_" + str(tmp_contrast)] = [tmp_id]

In [29]:
# 1.) Setup all the data needed for the plot ------------------------------
source_selected_contrast_grid = ColumnDataSource(contrast_curve_selected)
source_all_contrast_grids = ColumnDataSource(data=contrast_grids_prepared)

source_all_residual = ColumnDataSource(data=all_residual_prepared)
source_selected_residual = ColumnDataSource(data=residual_selected)

source_lookup = ColumnDataSource(data=idx_lookup_table)

# 2. Residual Plot  ----------------------------------------------------------
plot_residual = figure(
    width=320, height=300, 
    toolbar_location='left',
    tools="pan,wheel_zoom,reset,save",
    active_scroll="wheel_zoom",
    title="Fake planet residual")

plot_residual.x_range.range_padding = 0
plot_residual.y_range.range_padding = 0
plot_residual.xaxis.visible = False
plot_residual.yaxis.visible = False
plot_residual.title.text_font_size = '12pt'

min_c = np.min(residuals)
max_c = np.max(residuals)

color_mapper = LinearColorMapper(
    palette="Viridis256",
    low=min_c/3, high=max_c/4)

image = plot_residual.image(
    image="residual",
    color_mapper=color_mapper,
    x=0, y=0, 
    dw=residual_size, 
    dh=residual_size,
    source=source_selected_residual)

# 3. Contrast Grid Plot  ------------------------------------------------------
plot_contrast_grid = figure(
    title="Contrast Grid",
    x_range=(x_range_min, x_range_max), 
    y_range=(y_range_max, y_range_min),
    width=500, height=300,
    x_axis_label='Separation [FWHM]',
    y_axis_label='Planet-to-star flux ratio [mag]',
    tools = ["tap"],
    toolbar_location=None)

plot_contrast_grid.axis.major_label_text_font_size = "10px"
plot_contrast_grid.axis.major_label_standoff = 2
plot_contrast_grid.title.text_font_size = '12pt'

mapper = LinearColorMapper(
    palette="Viridis256",
    low=2, 
    high=8)

cgrid = plot_contrast_grid.rect(
    x="separation [FWHM]", 
    y="mag",
    width=1, 
    height=0.5,
    source=source_selected_contrast_grid,
    fill_color={'field': 'fpf', 'transform': mapper},
    nonselection_fill_alpha=1,
    selection_fill_alpha=1,
    selection_line_color="firebrick",
    selection_line_alpha=1,
    selection_line_width=5,
    line_color=None)

plot_contrast_grid.text(
    x="separation [FWHM]", 
   y="mag", 
   text="fpf_str",
   text_font_size="15px",
   selection_text_alpha = 1,
   nonselection_text_alpha = 1,
   text_align="center",
   text_baseline="middle",
   text_color="white",
   source=source_selected_contrast_grid,)

# Add the colorbar ---------------------------------------------------------
color_bar = ColorBar(color_mapper=mapper,
                     major_label_text_font_size="10px",
                     label_standoff=6, border_line_color=None)

color_bar_plot = figure(
    title=r"\[\text{ Detection uncertainty - }\sigma_{ N }\]",
    title_location="right", 
    height=300, 
    width=5,
    toolbar_location=None,
    min_border=0, 
    outline_line_color=None)

color_bar_plot.add_layout(color_bar, 'right')
color_bar_plot.title.align="center"
color_bar_plot.title.text_font_size = '10pt'

# 4.) Widgets ---------------------------------------------------------------
color_slider = RangeSlider(
    width=200,
    start=min_c, end=max_c, step=.1, 
    value=(min_c/3, max_c/4), 
    title="color range")

color_slider.js_link("value", color_mapper, "low", attr_selector=0)
color_slider.js_link("value", color_mapper, "high", attr_selector=1)

# Separation
separation_select = Select(
    title="Separation [FWHM]:", 
    width=170,
    value=separation_start, 
    options=separations)

# Contrast
contrast_select = Select(
    title="Planet-to-star flux ratio [mag]:", 
    width=200,
    value=contrast_start, 
    options=contrasts)

# PCA components
components_select = Select(
    title="PCA components", 
    width=170,
    value=component_start, 
    options=components)

# Fake planet idx
fake_planet_select = Select(
    width=100,
    title="Fake Planet ID", 
    value="a", 
    options=["a", "b", "c"])


# 5.) Interaction ------------------------------------------------------------
code_index = """
    //alert("Start") // For debugging
    // 1.) Check which grid value is selected
    const current_idx = source_selected_contrast_grid.selected.indices[0];
    const current_idx_str = String(current_idx);
    const current_id = components.value + "_" + fake_planet_id.value + "_" + current_idx_str;
    
    // 2.) Change the Select Widgets
    const tmp_separation = source_selected_contrast_grid.data["separation [FWHM]"][current_idx];
    const tmp_constrast = source_selected_contrast_grid.data["mag"][current_idx];
    
    separation.value = String(tmp_separation.toFixed(1));
    contrast.value = String(tmp_constrast.toFixed(1));
    
    // 3.) Update the Contrast Grid
    source_selected_contrast_grid.data = {
        "mag" : source_all_contrast_grids.data[components.value + "_mag"],
        "fpf" : source_all_contrast_grids.data[components.value + "_fpf"],
        "fpf_str" : source_all_contrast_grids.data[components.value + "_fpf_str"],
        "separation [FWHM]" : source_all_contrast_grids.data[components.value + "_separation [FWHM]"]}
    
    // 4.) Update the residual
    source_selected_residual.data = {"residual": [source_all_residual.data[current_id],]};
    
    //alert("Finish") // For debugging
"""

callback_selected = CustomJS(
    args=dict(
        source_all_contrast_grids=source_all_contrast_grids, 
        source_selected_contrast_grid = source_selected_contrast_grid,
        source_all_residual = source_all_residual,
        source_selected_residual = source_selected_residual,
        source_lookup=source_lookup,
        separation=separation_select,
        contrast = contrast_select,
        components = components_select,
        fake_planet_id = fake_planet_select),
    code=code_index)

code_widget = """
    const current_lookup = String(separation.value) + "_" + String(contrast.value)
    const current_id = source_lookup.data[current_lookup][0]
    source_selected_contrast_grid.selected.indices = [current_id]
"""

callback_widget_select = CustomJS(
    args=dict(
        source_selected_contrast_grid = source_selected_contrast_grid,
        source_lookup=source_lookup,
        separation=separation_select,
        contrast = contrast_select),
    code=code_widget)

taptool = plot_contrast_grid.select(type=TapTool)
source_selected_contrast_grid.selected.js_on_change('indices', callback_selected)

components_select.js_on_change('value', callback_selected)
fake_planet_select.js_on_change('value', callback_selected)

separation_select.js_on_change('value', callback_widget_select)
contrast_select.js_on_change('value', callback_widget_select)


# Select the initial value 
source_selected_contrast_grid.selected.indices = [int(start_idx)]

# 6.) Create the Layout
left_side = column(
    row(plot_contrast_grid,
        color_bar_plot),
    row(separation_select, 
        contrast_select,
        components_select))

right_side = column(
    plot_residual,
    row(fake_planet_select,
        color_slider))

final_plot = row(right_side, left_side)

output_file("Contrast_grid.html")
save(final_plot)
#show(final_plot)

'/home/ipa/quanz/user_accounts/mbonse/2021_Metrics/50_code/applefy/docs/source/02_user_documentation/Contrast_grid.html'